In [1]:
import tempfile
import os
from sktime.datasets import load_from_tsfile_to_dataframe

# 创建临时TS文件
def create_mock_tsfile():
    content = """@problemName MockMultivariate
@timestamps false
@univariate false
@equalLength true
@dimensions 3
@seriesLength 4
@classLabel true 0 1
@data
(1.1,2.1,3.1,4.1),(1.2,2.2,3.2,4.2),(1.3,2.3,3.3,4.3):0
(5.1,6.1,7.1,8.1),(5.2,6.2,7.2,8.2),(5.3,6.3,7.3,8.3):1"""
    
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.ts')
    with open(tmp.name, 'w') as f:
        f.write(content)
    return tmp.name

filepath = create_mock_tsfile()
print(f"临时文件路径: {filepath}")

临时文件路径: /tmp/tmp2ay_gh73.ts


In [2]:
# 加载数据并检查结构
X, y = load_from_tsfile_to_dataframe(filepath)

print("\n=== 解析结果验证 ===")
print(f"数据形状: {X.shape}")  # 应输出 (2, 3)
print(f"标签: {y}")           # 应输出 [0, 1]

# 检查第一个样本
print("\n第一个样本:")
print(f"通道1: {X.iloc[0, 0].values}")  # 应输出 [1.1, 2.1, 3.1, 4.1]
print(f"通道2: {X.iloc[0, 1].values}")  # 应输出 [1.2, 2.2, 3.2, 4.2]
print(f"通道3: {X.iloc[0, 2].values}")  # 应输出 [1.3, 2.3, 3.3, 4.3]

# 检查DataFrame类型
print("\n数据类型验证:")
print(f"单元格类型: {type(X.iloc[0,0])}")  # 应输出 pandas.Series
print(f"通道数: {X.shape[1]}")          # 应等于@dimensions声明的3
print(f"时间步长: {len(X.iloc[0,0])}")  # 应等于@seriesLength声明的4

# 清理临时文件
os.unlink(filepath)

ValueError: could not convert string to float: '(1.1'

10.22问题记录：
原始模型的多通道貌似是用括号分开？虽然输出看没有括号像是展平 但是真直接展平  模型输入尺寸就不对；所以应该是括号分开，但是函数load_from_tsfile_to_dataframe又没法识别括号 这样没法转化数据类型 就一直报错  could not convert string to float: '(1.1'

In [6]:
##尝试冒号分割
import tempfile
import os
from sktime.datasets import load_from_tsfile_to_dataframe

def create_proper_tsfile():
    """创建符合sktime完整要求的TS文件"""
    content = """@problemName ProperLabelDemo
@timestamps false
@missing false
@univariate false
@equalLength true
@dimensions 2
@seriesLength 3
@classLabel true 0 1 2
@data
1.1,1.2,1.3:2.1,2.2,2.3:0
4.1,4.2,4.3:5.1,5.2,5.3:1
7.1,7.2,7.3:8.1,8.2,8.3:2"""
    
    tmp = tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.ts')
    tmp.write(content)
    tmp.close()
    return tmp.name

# 测试加载
filepath = create_proper_tsfile()
print(f"文件路径: {filepath}")
print("文件内容:")
with open(filepath) as f:
    print(f.read())

try:
    X, y = load_from_tsfile_to_dataframe(filepath)
    
    print("\n=== 解析成功 ===")
    print(f"数据形状: {X.shape}")  # 应输出 (3, 2)
    print(f"标签: {y}")          # 应输出 [0 1 2]
    
    print("\n第一个样本:")
    print(f"通道1: {X.iloc[0, 0].values.tolist()}")
    print(f"通道2: {X.iloc[0, 1].values.tolist()}")
    print(f"对应标签: {y[0]}")
    
finally:
    os.unlink(filepath)  # 清理临时文件

文件路径: /tmp/tmp9p0s0bem.ts
文件内容:
@problemName ProperLabelDemo
@timestamps false
@missing false
@univariate false
@equalLength true
@dimensions 2
@seriesLength 3
@classLabel true 0 1 2
@data
1.1,1.2,1.3:2.1,2.2,2.3:0
4.1,4.2,4.3:5.1,5.2,5.3:1
7.1,7.2,7.3:8.1,8.2,8.3:2

=== 解析成功 ===
数据形状: (3, 2)
标签: ['0' '1' '2']

第一个样本:
通道1: [1.1, 1.2, 1.3]
通道2: [2.1, 2.2, 2.3]
对应标签: 0


In [12]:
import numpy as np
import os
import random
from collections import defaultdict

def convert_npz_to_ts_by_subject(npz_files, output_dir, dataset_name="CHBMIT_EEG", test_subjects=3, random_state=42):
    """
    按被试划分训练/测试集的TS转换器
    参数：
        test_subjects: 测试集被试数量（不是比例）
    """
    # 初始化
    random.seed(random_state)
    np.random.seed(random_state)
    os.makedirs(output_dir, exist_ok=True)
    
    # === 第一步：按被试加载数据 ===
    subject_data = defaultdict(list)
    subject_ids = set()
    
    for npz_file in sorted(npz_files):
        try:
            with np.load(npz_file, allow_pickle=True) as data:
                subject_id = os.path.basename(npz_file).split('_')[1]  # 提取被试ID（如chb01）
                subject_ids.add(subject_id)
                windows = np.nan_to_num(data['windows'], nan=0.0)
                labels = data['labels'].astype(int)
                subject_data[subject_id].append({
                    'windows': windows.astype(np.float32),
                    'labels': labels
                })
            print(f"✅ 已加载被试 {subject_id} 的数据")
        except Exception as e:
            print(f"❌ 加载失败 {npz_file}: {str(e)}")

    # === 第二步：按被试划分数据集 ===
    all_subjects = sorted(subject_ids)  # 获取所有唯一被试ID
    np.random.shuffle(all_subjects)      # 随机打乱
    
    test_subjs = all_subjects[:test_subjects]          # 前N个被试作为测试集
    train_subjs = all_subjects[test_subjects:]         # 剩余作为训练集
    
    print(f"\n=== 数据集划分 ===")
    print(f"总被试数: {len(all_subjects)}")
    print(f"训练集被试 ({len(train_subjs)}): {', '.join(train_subjs)}")
    print(f"测试集被试 ({len(test_subjs)}): {', '.join(test_subjs)}")

    # === 第三步：合并同一被试的所有样本 ===
    def combine_subject_data(subjects):
        """合并指定被试的所有样本"""
        samples, labels = [], []
        for subj in subjects:
            for data in subject_data[subj]:
                samples.append(data['windows'])
                labels.append(data['labels'])
        return np.concatenate(samples), np.concatenate(labels)
    
    X_train, y_train = combine_subject_data(train_subjs)
    X_test, y_test = combine_subject_data(test_subjs)

    # === 第四步：保存TS文件 ===
    def format_value(x):
        """保证6位小数"""
        return f"{float(x):.6f}".replace("-0.000000", "0.000000")

    def save_ts_file(data, labels, file_suffix):
        """保存TS文件"""
        ts_path = os.path.join(output_dir, f"{dataset_name}_{file_suffix}.ts")
        error_count = 0
        
        with open(ts_path, 'w') as f:
            # 文件头
            f.write(f"""@problemName {dataset_name}
@timestamps false
@univariate false
@equalLength true
@dimensions {data.shape[1]}
@seriesLength {data.shape[2]}
@classLabel true 0 1
@data
""")
            # 数据行
            for i in range(len(data)):
                try:
                    # 每个通道的值用逗号分隔，通道间用冒号分隔
                    channels = [
                        ",".join([format_value(x) for x in data[i, ch]]) 
                        for ch in range(data.shape[1])
                    ]
                    line = ":".join(channels) + f":{int(labels[i])}"
                    f.write(line + "\n")
                except Exception as e:
                    error_count += 1
                    print(f"样本{i}保存失败: {str(e)}")
        
        return error_count

    # 执行保存
    print("\n=== 保存文件中 ===")
    train_errors = save_ts_file(X_train, y_train, "TRAIN")
    test_errors = save_ts_file(X_test, y_test, "TEST")

    # === 第五步：保存元数据 ===
    metadata_path = os.path.join(output_dir, f"{dataset_name}_METADATA.txt")
    with open(metadata_path, 'w') as f:
        f.write(f"""# {dataset_name} 数据集元数据
## 被试划分
- 总被试数: {len(all_subjects)}
- 训练集被试: {len(train_subjs)} ({', '.join(train_subjs)})
- 测试集被试: {len(test_subjs)} ({', '.join(test_subjs)})

## 数据统计
- 训练样本数: {len(X_train)} (保存错误: {train_errors})
- 测试样本数: {len(X_test)} (保存错误: {test_errors})
- 通道数: {X_train.shape[1]}
- 时间步长: {X_train.shape[2]}

## 标签说明
- 0: 非癫痫发作
- 1: 癫痫发作

生成时间: {np.datetime64('now')}
""")

    print(f"""
=== 转换完成 ===
输出文件:
{output_dir}/
├── {dataset_name}_TRAIN.ts
├── {dataset_name}_TEST.ts
└── {dataset_name}_METADATA.txt

数据集统计:
- 训练集: {len(X_train)} 样本 (来自 {len(train_subjs)} 个被试)
- 测试集: {len(X_test)} 样本 (来自 {len(test_subjs)} 个被试)
""")

# 使用示例
if __name__ == "__main__":
    # 输入文件列表（修改为你的实际路径）
    input_files = [
        '/root/autodl-tmp/balanced_data/balanced_chb19.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb02.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb23.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb01.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb03.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb21.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb20.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb18.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb14.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb13.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb10.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb09.npz',
        '/root/autodl-tmp/balanced_data/balanced_chb05.npz'
    ]
    
    # 输出目录（建议绝对路径）
    output_dir = "/root/autodl-tmp/Time-Series-Library/dataset"
    dataset_name = "CHBMIT_EEG_BySubject"
    
    # 执行转换（测试集使用3个被试）
    convert_npz_to_ts_by_subject(
        npz_files=[f for f in input_files if os.path.exists(f)],
        output_dir=output_dir,
        dataset_name=dataset_name,
        test_subjects=3,  # 指定测试集被试数量
        random_state=42
    )

✅ 已加载被试 chb01.npz 的数据
✅ 已加载被试 chb02.npz 的数据
✅ 已加载被试 chb03.npz 的数据
✅ 已加载被试 chb05.npz 的数据
✅ 已加载被试 chb09.npz 的数据
✅ 已加载被试 chb10.npz 的数据
✅ 已加载被试 chb13.npz 的数据
✅ 已加载被试 chb14.npz 的数据
✅ 已加载被试 chb18.npz 的数据
✅ 已加载被试 chb19.npz 的数据
✅ 已加载被试 chb20.npz 的数据
✅ 已加载被试 chb21.npz 的数据
✅ 已加载被试 chb23.npz 的数据

=== 数据集划分 ===
总被试数: 13
训练集被试 (10): chb18.npz, chb10.npz, chb03.npz, chb02.npz, chb23.npz, chb09.npz, chb14.npz, chb20.npz, chb05.npz, chb13.npz
测试集被试 (3): chb21.npz, chb19.npz, chb01.npz

=== 保存文件中 ===

=== 转换完成 ===
输出文件:
/root/autodl-tmp/Time-Series-Library/dataset/
├── CHBMIT_EEG_BySubject_TRAIN.ts
├── CHBMIT_EEG_BySubject_TEST.ts
└── CHBMIT_EEG_BySubject_METADATA.txt

数据集统计:
- 训练集: 2629 样本 (来自 10 个被试)
- 测试集: 378 样本 (来自 3 个被试)

